# 1. Minimização

Pretende-se minimizar a seguinte função:

$$ y = f(x) = (x-2)^2+x^4 $$

**Pesquisa unidimensional**
- Método dos terços
- Secção Aurea
- Método da interpolação quadrática

**Pesquisa multidimensional**
- Método de Powell
- Método do gradiente
- Método da quadrática

## Calcular minimo de uma função utilizando um guess

-> start com guess

1. pesquisa para enquadrar num intervalo
    - (esq. ou dir.)
    - passo constante ou variavel  
2. redução intervalar
    - terços
    - secção aurea  
3. ajuste da quádrica

In [1]:
def function1(x):
    return (x-2)**2+x**4

In [2]:
def sequentialSearch(x0,func,h=1):
    a = x0
    b = a + h
    while func(a) > func(b):
        a += h
        b += h
    return [a,b]

int = sequentialSearch(0,function1,0.01)
print(int)

[0.8400000000000005, 0.8500000000000005]


Ficamos a saber que o minimo está entre 0.84 e 0.85

Passando agora à redução intervalar utilizando os dois métodos:

In [3]:
# Método dos Terços

def tercos(interval, f, prec = 0.0001):
    a = interval[0]
    b = interval[1]
    while abs(b-a) > prec:
        c = a + (b-a)/3.0
        d = b - (b-a)/3.0
        if f(c) < f(d):
            b = d
        else:
            a = c
    return [a,b]


# Método da secção aurea
from math import sqrt

def aureaSec(rangeVal, f, prec=0.1):
    # B = proporção
    B = (sqrt(5) - 1) / 2

    a = rangeVal[0]
    b = rangeVal[1]

    c = a + (b - a) * B
    d = b - (b - a) * B

    fc = f(c)
    fd = f(d)

    while abs(b - a) > prec:
        if fc < fd:
            b = d
            d = c
            fd = fc
            c = a + B * (b - a)
            fc = f(c)
        else:
            a = c
            c = d
            fc = fd
            d = b - B * (b - a)
            fd = f(d)

    return [a, b]

int_ter = tercos(int,function1,0.01)
int_sa = aureaSec(int,function1,0.01)

print("Intervalo com terços (prec = 0.01):",int_ter)
print("Intervalo com s.aurea (prec = 0.01):",int_sa)

Intervalo com terços (prec = 0.01): [0.8400000000000005, 0.8466666666666672]
Intervalo com s.aurea (prec = 0.01): [0.8461803398874995, 0.8500000000000005]


Agora façamos um ajuste da quadrática para obter o minimo

In [4]:
def quadAdjust(points, f):
    x1 = points[0]
    x3 = points[1]
    x2 = (x1 + x3) / 2.0

    h = x2 - x1

    return x2 - (h * (f(x1) - f(x3))) / (2 * (f(x1) - 2 * f(x2) + f(x3)))

min_ter = quadAdjust(int_ter, function1)
min_sa = quadAdjust(int_sa, function1)

print("Min (terços):",min_ter)
print("Min (s.aurea):",min_sa)

Min (terços): 0.8514833018671916
Min (s.aurea): 0.8608989841010513


# 2. Integração

O comprimento L do arco de uma curva de equação $ y = f(x) $ entre as abcissas $x=a$ e $x=b$ é dado por 

$$ L = \int_{a}^{b} \sqrt{1+(y')^2} dx $$

Pretende-se determinar o comprimento da curva $ y=e^{kx}$ ente $ x = a $ e $x = b$, recorrendo aos métodos de **Simpson** e dos **Trapézios**.

Partindo dos seguintes dados:
$$ k = 2.5 $$
$$ a = 0 $$
$$ b = 1 $$
$$ h = 0.125 $$

Antes de mais, devemos calcular a derivada, $ (e^{kx})' = ke^{x} $, portanto:

In [5]:
from math import sqrt, exp

def func(x):
    return sqrt(1+(2.5*exp(x*2.5))**2)

In [6]:
def trapRule(x0,x1,f,h=0.125):
    N = int((x1-x0)/h)
    x = x0 + h
    res = 0
    for i in range(N-1):
        res += f(x)
        x += h
    return h*(f(x0)+f(x1)+2*res)/2


def simpsonRule(x0,x1,f,h=0.125):
    res = 0
    N = int((x1-x0)/h)
    x = x0 +h
    for i in range(1,N):
        if i % 2 == 0:
            res += 2*f(x)
        else:
            res += 4*f(x)
        x += h
    
    return h*(f(x0)+f(x1)+res)/3

trap = trapRule(0,1,func)
simp = simpsonRule(0,1,func)

print("Trap:",trap)
print("Simp:",simp)

TypeError: 'list' object is not callable

In [ ]:
def qc(x0,x1,f,h,method):
    s = method(x0,x1,f,h)
    s1 = method(x0,x1,f,h/2)
    s2 = method(x0,x1,f,h/4)
    
    print("Method:",method.__name__)
    print("h=%.05f: %.05f" % (h,s))
    print("h=%.05f: %.05f" % (h/2,s1))
    print("h=%.05f: %.05f" % (h/4,s2))
    
    conv = (s1 - s) / (s2 - s1)
    order = round(sqrt(conv))
    error = (s2-s1)/(order**2 - 1)
    print("QC:",conv)
    print("Order:",order)
    print("Rel. Error: %0.05f\n" % error)
    
qc(0,1,func,0.125,trapRule)
qc(0,1,func,0.125,simpsonRule)
    

# 3. Zeros de Funções Reais

Pretendemos resolver a seguinte equação numericamente:

$$ e^{x} - x - 5 = 0 $$

Com as seguintes formulas de recorrencia para Picard-Peano:

$$ (1) x_{n+1} = e^{x_n}-5 \\ (2) x_{n+1} = ln(5+x_n) $$

In [ ]:
import matplotlib.pyplot as plt
from math import log, exp
from numpy import arange

def func3(x):
    return exp(x)-x-5

x_list = [i for i in arange(-6,4,0.1)]
y_list = [func3(i) for i in arange(-6,4,0.1)]

plt.plot(x_list,y_list)
plt.axhline(0, color='black')

Como podemos ver com o gráfico, a função tem 2 zeros, um no intervalo $[-6,-4]$ e outro no intervalo $[1,3]$.

A primeira formula de recorrencia vai dar-nos a raiz negativa, uma vez que a derivada nesse intervalo é inferior a 1. Pela mesma razão a segunda formula vai dar-nos a raiz positiva.

## Picard-Peano

Podemos utilizar a precisão absoluta como critério de paragem, sendo assim, a formula é sempre a mesma.

Para calcular a maior raiz:

In [ ]:
def rec1Func3(x):
    return exp(x) - 5

def rec2Func3(x):
    return log(5+x)


def picardPeano(guess, rec, iterations=10):
    # print("Iteration 0: %.05f" % guess)
    res = [guess]
    for i in range(1,iterations):
        guess = rec(guess)
        # print("Iteration %d: %.05f" % (i,guess))
        res.append(guess)
        
    return res

In [ ]:
def diffFunc3(x):
    return exp(x)-1

def newton(guess,func,diff,iterations=10):
    # print("Iteration 0: %.05f" % guess)
    res = [guess]
    for i in range(1,iterations):
        guess -= func(guess)/diff(guess)
        # print("Iteration %d: %.05f" % (i,guess))
        res.append(guess)
        
    return res

ppeano = picardPeano(3,rec2Func3)
newt = newton(3,func3,diffFunc3)

print("Picard-Peano\tNewton")
print("-----------------------")
for i,c in enumerate(ppeano):
    print("   Iteration",i)
    print("%.05f        %.05f" % (c, newt[i]))
    

Podemos verificar que o método de Newton convergeu logo na 4ª iteração enquanto que o método de Picard-Peano só converge a partir da 6ª iteração.

# 4. Sistemas de Eq. Diferenciais

$\frac{dC}{dt} = -e^{\frac{-0.5}{T+273}} * C $  
$\frac{dT}{dt} = 30e^{\frac{-0.5}{T+273}}*C -0.5*(T-20) $

$ t = 0 $  
$ C = 2.5 $  
$ T = 25 $

In [ ]:
from math import exp

def dC(t,C,T):
    return -exp(-0.5/(T+273))*C

def dT(t,C,T):
    return 30*exp(-0.5/(T+273))*C - 0.5*(T-20)

def euler(deltaX, x, y, z, xf, dy, dz,info=True):
    if info == True:    
        print("Iteration 0")
        print("x: %.05f" % x)
        print("y: %.05f" % y)
        print("z: %.05f" % z)
    i = 1
    while x < xf:
        x += deltaX
        deltaY = dy(x, y, z)
        deltaZ = dz(x, y, z)
        y += deltaY * deltaX
        z += deltaZ * deltaX
        
        if info:
            print("Iteration", i)
            print("x: %.05f" % x)
            print("y: %.05f" % y)
            print("z: %.05f" % z)
        i += 1
        
    return y, z

def runga_kutta_4(deltaX, x, y, z, xf, dy, dz, info=True):
        
    if info:
        print("Iteration 0")
        print("x: %.05f" % x)
        print("y: %.05f" % y)
        print("z: %.05f" % z)
    i = 1
    while x < xf:
        x += deltaX
        dY1 = dy(x, y, z) * deltaX
        dZ1 = dz(x, y, z) * deltaX
        dY2 = deltaX * dy(x + deltaX / 2, y + dY1 / 2, z + dZ1 / 2)
        dZ2 = deltaX * dz(x + deltaX / 2, y + dY1 / 2, z + dZ1 / 2)
        dY3 = deltaX * dy(x + deltaX / 2, y + dY2 / 2, z + dZ2 / 2)
        dZ3 = deltaX * dz(x + deltaX / 2, y + dY2 / 2, z + dZ2 / 2)
        dY4 = deltaX * dy(x + deltaX, y + dY3, z + dZ3)
        dZ4 = deltaX * dz(x + deltaX, y + dY3, z + dZ3)
        y += (dY1 / 6 + dY2 / 3 + dY3 / 3 + dY4 / 6)
        z += (dZ1 / 6 + dZ2 / 3 + dZ3 / 3 + dZ4 / 6)
        
        if info:
            print("Iteration", i)
            print("x: %.05f" % x)
            print("y: %.05f" % y)
            print("z: %.05f" % z)
        i += 1
    
    return y, z


def convergence_quotient(deltaX, x, y, z, xf, dy, dz, method,info=False):
    print(method.__name__)
    y0, z0 = method(deltaX, x, y, z, xf, dy, dz, info)
    print("h: %.05f" % deltaX)
    print("y: %.05f" % y0)
    print("z: %.05f" % z0)
    y1, z1 = method(deltaX / 2, x, y, z, xf, dy, dz, info)
    print("\nh: %.05f" % (deltaX / 2))
    print("y: %.05f" % y1)
    print("z: %.05f" % z1)
    y2, z2 = method(deltaX / 4, x, y, z, xf, dy, dz, info)
    print("\nh: %.05f" % (deltaX / 4))
    print("y: %.05f" % y2)
    print("z: %.05f" % z2)

    quotient_y = (y1 - y0) / (y2 - y1)
    quotient_z = (z1 - z0) / (z2 - z1)

    error_y = (y2 - y1) / (round(quotient_y) - 1)
    error_z = (z2 - z1) / (round(quotient_z) - 1)

    print("\nCQ(y): %.05f" % quotient_y)
    print("Error(y): %.05f" % error_y)

    print("CQ(z): %.05f" % quotient_z)
    print("Error(z): %.05f" % error_z)

    return quotient_y, quotient_z



print("Primeira alinea")
euler(0.25,0,2.5,25,0.5,dC,dT)
print("\nSegunda Alinea")
runga_kutta_4(0.25,0,2.5,25,0.5,dC,dT)

print("\nTerceira Alinea")
convergence_quotient(0.25,0,2.5,25,0.5,dC,dT,euler)